In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

from geo_dist_prep.data import GEONAMES_DB
from geo_dist_prep.schemas.geoname import GeoName

engine = create_engine(f"sqlite:///../{GEONAMES_DB}")

Session = sessionmaker(bind=engine)
session = Session()


In [ ]:
import pandas as pd

from sqlalchemy.sql import and_, select
from sqlalchemy import func, bindparam, Integer, text, Float

from geo_dist_prep.schemas.helpers import *

southport = 26700924
lon, lat = -3.004175, 53.647599
#lon, lat = -3.048624, 53.603828

dist = 100

# # get all places within 4km of southport
nearby_places = (
     session.query(GeoName)
     .filter(nearby(lat, lon, dist))
     .add_columns(distance(GeoName.lat, GeoName.lon, lat, lon))
     .add_columns(grid_coord(GeoName, 1).label("grid_coord"))
     .order_by(distance(GeoName.lat, GeoName.lon, lat, lon))
)

q = nearby_places.statement.compile()
df = pd.read_sql_query(q, session.bind)

df



In [ ]:
import matplotlib.pyplot as plt

dist = 350
grid_size = 32


nearby_places = (
     session.query(GeoName)
     .filter(nearby(lat, lon, dist))
     .add_columns(distance(GeoName.lat, GeoName.lon, lat, lon))
     .add_columns(grid_coord(GeoName, grid_size).label("grid_coord"))
     .order_by(distance(GeoName.lat, GeoName.lon, lat, lon))
)

q = nearby_places.statement.compile()
df = pd.read_sql_query(q, session.bind)

plt.scatter(df.lon, df.lat, s=1, color='blue')
plt.xlim(min(df.lon), max(df.lon))
plt.ylim(min(df.lat), max(df.lat))

grid_column = grid_coord(GeoName, grid_size).label('grid_coord')
distinct_by_grid = nearby_places.order_by(GeoName.place_rank + 1 / func.random()).group_by('grid_coord').distinct()
q = distinct_by_grid.statement.compile()
df2 = pd.read_sql_query(q, session.bind)

plt.scatter(df2.lon, df2.lat, s=2, color='red')
df2

In [ ]:
from sqlalchemy import update

for grid_size in (16, 32):
    grid_column = grid_coord(GeoName, grid_size).label('grid_coord')
    table_plus_grid = session.query(GeoName).add_columns(grid_column)
    by_importance = table_plus_grid.order_by(GeoName.importance)
    distinct = by_importance.group_by('grid_coord').distinct().subquery()

    update_stmt = (
        update(GeoName).values(tree_depth=grid_size)
        .where(GeoName.osm_id == distinct.c.osm_id)
    )

session.execute(update_stmt)